<font size=6>

<b>Curso de Introducción a la Programación en Python</b>
</font>

<font size=4>
    
Curso de formación interna, CIEMAT 
Junio, 2020

Antonio Delgado Peris
</font>

some url.. 

<br/>
<br/>

# Tema 8 - Otros recursos

## Objetivos

- Aprender a usar y gestionar las excepciones en Python

- Introducir/profundizar en algunos recursos sintácticos _avanzados_

  - _Comprenhensions_
  - Decoradores
  - Iteradores y generadores
  - _Context managers_
  

- Conocer algunos módulos de la librería estándar

  - re
  - argparse
  - subprocess
  - datetime
  - os, pathlib, sys
  - threading/multiprocessing
  - pytest